In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import tensorflow
from tensorflow import keras
from keras import utils
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Conv2D,BatchNormalization,MaxPooling2D,Flatten,Dropout
from keras.applications.vgg16 import VGG16

/Users/rahulchauhan/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
import os
import cv2

def load_images_and_labels(directory, label):
    images = []
    labels = []
    for filename in os.listdir(directory):
        if filename.endswith(".jpg"):
            img_path = os.path.join(directory, filename)
            img = cv2.imread(img_path)
            img = cv2.resize(img, (128, 128)) # Resize for consistency
            images.append(img)
            labels.append(label)
    return images, labels


# Define directories
men_dir = '/Users/rahulchauhan/Downloads/dataset-3/MEN'
women_dir = '/Users/rahulchauhan/Downloads/dataset-3/WOMAN'

# Load images
men_images, men_labels = load_images_and_labels(men_dir, 0)
women_images, women_labels = load_images_and_labels(women_dir, 1)


Invalid SOS parameters for sequential JPEG


In [3]:
data = np.array(men_images + women_images)
labels = np.array(men_labels + women_labels)

In [5]:
from tensorflow.keras.utils import to_categorical


In [6]:
images = data/255.0
labels = to_categorical(labels,num_classes = 2)

In [19]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

In [7]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(images,labels,test_size=0.2,random_state=42)

In [12]:
model = Sequential()
input_size = (128,128,3)
model.add(Conv2D(32,kernel_size=3,padding='valid',activation='relu',input_shape = input_size))
model.add(Dropout(0.2))
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='same'))

model.add(Conv2D(64,kernel_size=3,padding='same',activation='relu'))
model.add(Dropout(0.2))
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='same'))

model.add(Conv2D(128,kernel_size=3,padding='same',activation='relu'))
model.add(Dropout(0.2))
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='same'))

model.add(Flatten())

model.add(Dense(128,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(2,activation='softmax'))

model.compile(optimizer = 'adam',loss = 'categorical_crossentropy',metrics = ['accuracy'])

In [13]:
history = model.fit(x_train,y_train,epochs=10,validation_data=(x_test,y_test))

Epoch 1/10
30/30 [==============================] - 6s 202ms/step - loss: 0.9125 - accuracy: 0.5554 - val_loss: 0.6594 - val_accuracy: 0.6109
Epoch 2/10
30/30 [==============================] - 6s 193ms/step - loss: 0.5761 - accuracy: 0.6998 - val_loss: 0.5946 - val_accuracy: 0.6402
Epoch 3/10
30/30 [==============================] - 6s 208ms/step - loss: 0.4948 - accuracy: 0.7626 - val_loss: 0.5889 - val_accuracy: 0.7113
Epoch 4/10
30/30 [==============================] - 6s 200ms/step - loss: 0.4551 - accuracy: 0.7908 - val_loss: 0.5735 - val_accuracy: 0.7197
Epoch 5/10
30/30 [==============================] - 6s 202ms/step - loss: 0.4016 - accuracy: 0.8274 - val_loss: 0.5543 - val_accuracy: 0.7364
Epoch 6/10
30/30 [==============================] - 6s 200ms/step - loss: 0.3320 - accuracy: 0.8567 - val_loss: 0.5346 - val_accuracy: 0.7448
Epoch 7/10
30/30 [==============================] - 6s 207ms/step - loss: 0.2680 - accuracy: 0.8881 - val_loss: 0.5446 - val_accuracy: 0.7490
Epoch 

In [17]:
y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

8/8 [==============================] - 0s 39ms/step


In [21]:
classification_report(y_true,y_pred_classes)

'              precision    recall  f1-score   support\n\n           0       0.76      0.83      0.79       134\n           1       0.75      0.67      0.71       105\n\n    accuracy                           0.76       239\n   macro avg       0.76      0.75      0.75       239\nweighted avg       0.76      0.76      0.76       239\n'

In [27]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.utils import to_categorical


In [28]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)  # Add a fully connected layer
x = Dense(2, activation='softmax')(x)  # Add a final softmax layer for binary classification

# Define the new model
model = Model(inputs=base_model.input, outputs=x)

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [29]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 128, 128, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 128, 128, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 64, 64, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 64, 64, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 64, 64, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 32, 32, 128)       0     

In [30]:
history = model.fit(x_train,y_train,epochs = 10,validation_data=(x_test,y_test))

Epoch 1/10
30/30 [==============================] - 35s 1s/step - loss: 0.4906 - accuracy: 0.7374 - val_loss: 0.3034 - val_accuracy: 0.8787
Epoch 2/10
30/30 [==============================] - 34s 1s/step - loss: 0.2969 - accuracy: 0.8818 - val_loss: 0.2374 - val_accuracy: 0.9079
Epoch 3/10
30/30 [==============================] - 37s 1s/step - loss: 0.2386 - accuracy: 0.9153 - val_loss: 0.2062 - val_accuracy: 0.9038
Epoch 4/10
30/30 [==============================] - 41s 1s/step - loss: 0.1969 - accuracy: 0.9310 - val_loss: 0.1851 - val_accuracy: 0.9205
Epoch 5/10
30/30 [==============================] - 38s 1s/step - loss: 0.1749 - accuracy: 0.9320 - val_loss: 0.1818 - val_accuracy: 0.9205
Epoch 6/10
30/30 [==============================] - 42s 1s/step - loss: 0.1541 - accuracy: 0.9414 - val_loss: 0.1736 - val_accuracy: 0.9289
Epoch 7/10
30/30 [==============================] - 49s 2s/step - loss: 0.1390 - accuracy: 0.9550 - val_loss: 0.1724 - val_accuracy: 0.9247
Epoch 8/10
30/30 [==